In [62]:
#Import libraries
import re
import random

In [63]:
#Define global variables

# Visual to logical mapping (letters to state indices)
visual_to_logical = {'A': 9, 'B': 9, 'C': 9, 'D': 9, 'E': 4, 'F': 9, 'G': 9, 'H': 9, 'I': 9}

# Game state string: 'x', 'o', or '-'
state = '---------'

# Turn counter: even = program (x), odd = opponent (o)
turn = 0

# Regex win states for terminal checks
win_patterns = {
    'xxx......': 'x', '...xxx...': 'x', '......xxx': 'x',
    'x..x..x..': 'x', '.x..x..x.': 'x', '..x..x..x': 'x',
    'x...x...x': 'x', '..x.x.x..': 'x',
    'ooo......': 'o', '...ooo...': 'o', '......ooo': 'o',
    'o..o..o..': 'o', '.o..o..o.': 'o', '..o..o..o': 'o',
    'o...o...o': 'o', '..o.o.o..': 'o'
}

# Journal of state transitions for learning
state_history = []

In [64]:
#Define functions

def orient_board(first_space):
    if first_space in ['A', 'B']:
        visual_to_logical.update({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8})
    elif first_space in ['C', 'F']:
        visual_to_logical.update({'A': 6, 'B': 3, 'C': 0, 'D': 7, 'E': 4, 'F': 1, 'G': 8, 'H': 5, 'I': 2})
    elif first_space in ['I', 'H']:
        visual_to_logical.update({'A': 8, 'B': 7, 'C': 6, 'D': 5, 'E': 4, 'F': 3, 'G': 2, 'H': 1, 'I': 0})
    elif first_space in ['G', 'D']:
        visual_to_logical.update({'A': 2, 'B': 5, 'C': 8, 'D': 1, 'E': 4, 'F': 7, 'G': 0, 'H': 3, 'I': 6})

def display_board():
    values = {k: '-' if visual_to_logical[k] == 9 else state[visual_to_logical[k]] for k in visual_to_logical}
    print('======================')
    print(f"  A: {values['A']} | B: {values['B']} | C: {values['C']}")
    print('-------|------|-------')
    print(f"  D: {values['D']} | E: {values['E']} | F: {values['F']}")
    print('-------|------|-------')
    print(f"  G: {values['G']} | H: {values['H']} | I: {values['I']}")
    print('======================')

def make_move():
    global state
    if (turn % 2) == 0:
        print("Program's move...")
        previous = state
        state = choose_move()
        state_history.append((previous, state))
        if turn == 0 and state != '----x----':
            orient_board('A')
    else:
        display_board()
        while True:
            player_input = input("Your move: ").upper()
            if (player_input in 'ABCDEFGHI' and len(player_input) == 1 and
                (visual_to_logical[player_input] == 9 or state[visual_to_logical[player_input]] == '-')):
                break
            print("You can't move there! Try again.")
        if visual_to_logical[player_input] == 9:
            orient_board(player_input)
        idx = visual_to_logical[player_input]
        state = state[:idx] + 'o' + state[idx+1:]

def choose_move():
    if turn == 8:
        return state.replace('-', 'x')
    options = STM[turn // 2][state]
    r = random.random()
    cumulative = 0.0
    for outcome in options:
        cumulative += options[outcome]
        if r <= cumulative:
            return outcome
    print(f"choose_move(): Probabilities error on state {state}")
    return list(options.keys())[-1]

def update_learning(won):
    for i in range(min(len(state_history), 4)):
        prev_state, chosen_state = state_history[i]
        beta = betaReward if won else -betaPunish
        delta = (1.0 - STM[i][prev_state][chosen_state]) if won else STM[i][prev_state][chosen_state]
        num_options = len(STM[i][prev_state])
        for option in STM[i][prev_state]:
            if option == chosen_state:
                STM[i][prev_state][option] += beta * delta
            else:
                STM[i][prev_state][option] -= beta * delta / (num_options - 1)
    print('The program has learned from this experience :)' if won else 'The program has learned from this experience :(')

def get_branches(start):
    player = 'x' if start.count('x') == start.count('o') else 'o'
    return [start[:i] + player + start[i+1:] for i in range(len(start)) if start[i] == '-']

In [65]:
#Build the STM
STM = [{'---------': {'x--------': 1/3, '-x-------': 1/3, '----x----': 1/3}}]

#Build STM for turns 2, 4, 6
for _ in range(3):
    new_layer = {}
    for state_dict in STM[-1].values():
        for result in state_dict:
            starts = get_branches(result)
            for s in starts:
                if s.startswith('--') or any(re.search(p, s) for p in win_patterns):
                    continue
                ends = get_branches(s)
                prob = 1.0 / len(ends)
                new_layer[s] = {e: prob for e in ends}
    STM.append(new_layer)

print('STM assembled.')

STM assembled.


In [66]:
#Set beta values
betaReward = 0.5
betaPunish = 0.25

In [76]:
#Start a new game

#reset variables
visual_to_logical.update({k: 9 for k in visual_to_logical})
visual_to_logical['E'] = 4
state = '---------'
state_history = []
turn = 0
done = False

ans = input('Print the state Strings? (y/n): ')
print_states = ans.lower().startswith('y')
print('Taking that as a ' + ('yes' if print_states else 'no'))

print("\nLet's play Tic Tac Toe!\n~~~~~~~~~~~~~~~~~~~~~~~")

while True:
    if print_states:
        print('Current state: ' + state)
    for pattern in win_patterns:
        if re.search(pattern, state):
            print('\nGame over! ~~~~~~~~~~~')
            display_board()
            print(win_patterns[pattern] + ' wins!')
            update_learning(win_patterns[pattern] == 'x')
            done = True
            break
    if done:
        break
    if turn >= 9:
        print('\nGame over! ~~~~~~~~~~~')
        display_board()
        print("It's a tie!")
        update_learning(True)
        break
    make_move()
    turn += 1

Print the state Strings? (y/n):  n


Taking that as a no

Let's play Tic Tac Toe!
~~~~~~~~~~~~~~~~~~~~~~~
Program's move...
  A: x | B: - | C: -
-------|------|-------
  D: - | E: - | F: -
-------|------|-------
  G: - | H: - | I: -


Your move:  e


Program's move...
  A: x | B: x | C: -
-------|------|-------
  D: - | E: o | F: -
-------|------|-------
  G: - | H: - | I: -


Your move:  c


Program's move...
  A: x | B: x | C: o
-------|------|-------
  D: - | E: o | F: x
-------|------|-------
  G: - | H: - | I: -


Your move:  g



Game over! ~~~~~~~~~~~
  A: x | B: x | C: o
-------|------|-------
  D: - | E: o | F: x
-------|------|-------
  G: o | H: - | I: -
o wins!
The program has learned from this experience :(


In [78]:
#Show the STM
show_one_state = False # make true to only STM for one state
specific = '---------'  # a valid 9-char state before x's move

for i, layer in enumerate(STM):
    turn = i * 2
    if show_one_state and (specific.count('x') + specific.count('o')) != turn:
        continue
    print(f'Turn {turn} ====================================')
    for state in layer:
        if show_one_state and state != specific:
            continue
        print(f'{state}:')
        for result, prob in layer[state].items():
            print(f'    {result}: {prob}')


Turn 0 ====================================
---------:
    x--------: 0.28125
    -x-------: 0.296875
    ----x----: 0.421875
Turn 2 ====================================
xo-------:
    xox------: 0.14285714285714285
    xo-x-----: 0.14285714285714285
    xo--x----: 0.14285714285714285
    xo---x---: 0.14285714285714285
    xo----x--: 0.14285714285714285
    xo-----x-: 0.14285714285714285
    xo------x: 0.14285714285714285
x-o------:
    xxo------: 0.14285714285714285
    x-ox-----: 0.14285714285714285
    x-o-x----: 0.14285714285714285
    x-o--x---: 0.14285714285714285
    x-o---x--: 0.14285714285714285
    x-o----x-: 0.14285714285714285
    x-o-----x: 0.14285714285714285
x--o-----:
    xx-o-----: 0.14285714285714285
    x-xo-----: 0.14285714285714285
    x--ox----: 0.14285714285714285
    x--o-x---: 0.14285714285714285
    x--o--x--: 0.14285714285714285
    x--o---x-: 0.14285714285714285
    x--o----x: 0.14285714285714285
x---o----:
    xx--o----: 0.10714285714285714
    x-x-o----: 0